In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
interval_comment = "Alina"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-11-01')
train_start_time = pd.to_datetime('2014-03-31')
train_end_time = pd.to_datetime('2014-10-01')
test_end_time = pd.to_datetime('2014-11-01')


#infile_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
#infile_jobs_trainset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_trainset"
infile_jobs_testset = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_testset"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
#jobs_train = pd.read_csv(infile_jobs_trainset + ".csv", index_col=0)
#print("train set contains {} records".format(jobs_train.shape[0]))
jobs_test = pd.read_csv(infile_jobs_testset + ".csv", index_col=0)
print("test set contains {} records".format(jobs_test.shape[0]))


test set contains 18224 records


In [3]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))
### clean the data
# remove jobs runned on the inexistent node 129
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129] 
# remove jobs with the same id that runned on the same node
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("after the clean, jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

### merge the info from the two tables
merged_jobs_to_nodes = pd.merge(jobs_test, jobs_to_nodes_whole_data, how='left', on='job_id_string')
print("merged_jobs_to_nodes contains {} records".format(merged_jobs_to_nodes.shape[0]))


jobs_to_nodes_whole_data contains 469095 records
after the clean, jobs_to_nodes_whole_data contains 444610 records
merged_jobs_to_nodes contains 18751 records


In [ ]:
#outfile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#outfile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
#outfile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
#outfile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"

# ---- whole
#infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
#df_user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
#infile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
#df_user_job = pd.read_csv(infile_user_job + ".csv", index_col=0)
#infile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
#df_user_node = pd.read_csv(infile_user_node + ".csv", index_col=0)
#infile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"
#df_user = pd.read_csv(infile_user + ".csv", index_col=0)
#infile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole"
#df_node = pd.read_csv(infile_node + ".csv", index_col=0)


In [20]:
def validate_predictions(jobs_test, predictions):
    mean_real_pow = jobs_test['real_pow'].mean()
    print("the mean is {}".format(mean_real_pow))

    eq_num = (jobs_test['real_pow'] - predictions)**2
    eq_den = (jobs_test['real_pow'] - mean_real_pow)**2
    N = eq_num.shape[0]

    r_2 = 1 - (eq_num.sum() / eq_den.sum())
    print("r_2 is {}".format(r_2))

    rmse = math.sqrt(eq_num.sum() / N)
    print("RMSE is {}".format(rmse))
    print("NRMSE is {}".format(rmse/mean_real_pow))

## method 5 - Per node

In [24]:
# method 5 - Per node

infile_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_node_whole"
df_node = pd.read_csv(infile_node + ".csv", index_col=0)
df_node.head(5)


,node_id,mean_core,count
0,1,9.574410,667
1,2,8.739935,109
2,3,17.070914,806
3,4,16.378783,941
4,5,18.866071,368


In [14]:
jobs_test['pred_perNode'] = 0.0
i = 0

for job_index, job_row in jobs_test.iterrows():

    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_row['job_id_string']] 
    job_predicted_consumption = 0.0
    
    for index_node, job_to_node in job_to_nodes.iterrows():
        mean_core_consumption = df_node[df_node['node_id'] == job_to_node['node_id']].iloc[0]['mean_core']
        job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumption
        
    jobs_test.loc[job_index, 'pred_perNode'] = job_predicted_consumption
    if (i % 1500 == 0):
        print(i)
    i += 1


0
10000


In [ ]:
jobs_test.to_csv(infile_jobs_testset + ".csv")

In [21]:
validate_predictions(jobs_test, jobs_test['pred_perNode'])

the mean is 75.89084796065079
r_2 is 0.012775884204423327
RMSE is 178.44238184926567
NRMSE is 2.35130304436429


## method 4 - Per user

In [26]:
# method 4 - Per user

infile_user = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_whole"
df_user = pd.read_csv(infile_user + ".csv", index_col=0)
df_user.head(5)


,user,mean_core,count
0,a07smr01,16.320424,5694
1,a08tra01,28.721584,12
2,a08tra11,108.508580,3
3,a08tra35,14.327342,8
4,a08tra37,13.757758,1


In [29]:
jobs_test['pred_user'] = 0.0
i = 0
for job_index, job_row in jobs_test.iterrows():
    job_user = job_row['user']
    job_ncores = job_row['cpu_req']
    
    data_user = df_user[df_user['user'] == job_user]
    if(data_user.shape[0] != 0):     
        mean_core_consumptions = data_user.iloc[0]['mean_core']
        job_predicted_consumption = job_ncores * mean_core_consumptions
    else:
        job_predicted_consumption = -1
    
    jobs_test.loc[job_index, 'pred_user'] = job_predicted_consumption
    
    if (i % 1500 == 0):
        print(i)
    i += 1
    

0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000


In [33]:
job_to_test = jobs_test[jobs_test['pred_user'] != -1] # some users are unknown
print("{} / {}".format(job_to_test.shape[0], jobs_test.shape[0])
validate_predictions(job_to_test, job_to_test['pred_user'])

18218
the mean is 75.88775808441862
r_2 is 0.6232456417832886
RMSE is 110.25197148026652
NRMSE is 1.4528294716207146


## method 3 - Per user per node

In [66]:
infile_user_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_node_whole"
df_user_node = pd.read_csv(infile_user_node + ".csv", index_col=0)
df_user_node.head(5)


,user,node_id,mean_core,count
0,a07smr01,1,15.386652,5
1,a07smr01,2,18.405537,4
2,a07smr01,6,18.405537,4
3,a07smr01,7,28.234849,3
4,a07smr01,33,7.076942,40


In [67]:
jobs_test['pred_perUserNode'] = 0.0
i = 0
for job_index, job_row in jobs_test.iterrows():
    
    isMethod3Avaiable = True # if the user-node in unknown
    job_user = job_row['user']
    
    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_row['job_id_string']] 
    job_predicted_consumption = 0.0
    
    for index_node, job_to_node in job_to_nodes.iterrows():
        
        data_user = df_user_node[(df_user_node['node_id'] == job_to_node['node_id']) & (df_user_node['user'] == job_to_node['user'])]
        if(data_user.shape[0] != 0):     
            mean_core_consumptions = data_user.iloc[0]['mean_core']
            job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumptions
        else:
            isMethod3Avaiable = False
        
    if(isMethod3Avaiable):
        jobs_test.loc[job_index, 'pred_perUserNode'] = job_predicted_consumption
    else:
        jobs_test.loc[job_index, 'pred_perUserNode'] = -1
        
    if (i % 1500 == 0):
        print(i)
    i += 1

0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000


In [68]:
job_to_test = jobs_test[jobs_test['pred_perUserNode'] != -1] # some users are unknown
print("{} / {}".format(job_to_test.shape[0], jobs_test.shape[0]))
validate_predictions(job_to_test, job_to_test['pred_perUserNode'])

18112 / 18224
the mean is 74.44670747208764
r_2 is 0.7084425915406942
RMSE is 95.50883193952748
NRMSE is 1.282915459697619


## method 2 - Per user per job

In [55]:
infile_user_job = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_whole"
df_user_job = pd.read_csv(infile_user_job + ".csv", index_col=0)
df_user_job.head(5)


,user,job_name,mean_core,count
0,a07smr01,A2REPO,4.371550,76
1,a07smr01,ARCHIVE,11.895912,1
2,a07smr01,ARKI,4.513156,110
3,a07smr01,INIT,4.865106,95
4,a07smr01,LM28_OPE,22.266872,1164


In [52]:
jobs_test['pred_userJob'] = 0.0
i = 0
for job_index, job_row in jobs_test.iterrows():
    job_user = job_row['user']
    job_name = job_row['job_name']
    job_ncores = job_row['cpu_req']
    
    df = df_user_job[df_user_job['user'] == job_user]
    df = df[df['job_name'] == job_name]
    
    if(df.shape[0] != 0):
        mean_core_cons = df['mean_core']
        job_predicted_consumption = job_ncores * mean_core_cons
    else:
        job_predicted_consumption = -1
    
    jobs_test.at[job_index, 'pred_userJob'] = job_predicted_consumption
    
    if (i % 1500 == 0):
        print(i)
    i += 1

0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000


In [53]:
job_to_test = jobs_test[jobs_test['pred_userJob'] != -1] # some users are unknown
print("{} / {}".format(job_to_test.shape[0], jobs_test.shape[0]))
validate_predictions(job_to_test, job_to_test['pred_userJob'])

4377 / 18224
the mean is 113.31462449690635
r_2 is 0.863427950022667
RMSE is 119.56313330510604
NRMSE is 1.0551430041439205


## method 1 - Per user per job per node

In [54]:
infile_user_job_node = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_user_job_node_whole"
df_user_job_node = pd.read_csv(infile_user_job_node + ".csv", index_col=0)
df_user_job_node.head(5)


,user,job_name,node_id,mean_core,count
0,a07smr01,A2REPO,7,12.412156,1
1,a07smr01,A2REPO,33,3.937672,1
2,a07smr01,A2REPO,35,4.870611,53
3,a07smr01,A2REPO,46,2.749792,21
4,a07smr01,ARCHIVE,7,11.895912,1


In [64]:
jobs_test['pred_userJobNode'] = 0.0
i = 0
for job_index, job_row in jobs_test.iterrows():
    job_user = job_row['user']
    job_name = job_row['job_name']
    job_predicted_consumption = 0.0
    isMethod3Avaiable = True

    mean_core_user_job = df_user_job_node.loc[(df_user_job_node['user']==job_user) & (df_user_job_node['job_name'] == job_name)]
    if(mean_core_user_job.shape[0] != 0):  # if != 0 than user and job name exist on hystoric data
        
        job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_row['job_id_string']] 
    
        for index_node, job_to_node in job_to_nodes.iterrows():
            
            data_user = mean_core_user_job[mean_core_user_job['node_id'] == job_to_node['node_id']]
            if(data_user.shape[0] != 0):     
                mean_core_consumptions = data_user.iloc[0]['mean_core']
                job_predicted_consumption += job_to_node['ncpus'] * mean_core_consumptions
            else:
                isMethod3Avaiable = False
    else:
        isMethod3Avaiable = False
    
    if(isMethod3Avaiable):
        jobs_test.loc[job_index, 'pred_userJobNode'] = job_predicted_consumption
    else:
        jobs_test.loc[job_index, 'pred_userJobNode'] = -1
        
    if (i % 1500 == 0):
        print(i)
    i += 1

0
1500
3000
4500
6000
7500
9000
10500
12000
13500
15000
16500
18000


In [65]:
job_to_test = jobs_test[jobs_test['pred_userJobNode'] != -1] # some users are unknown
print("{} / {}".format(job_to_test.shape[0], jobs_test.shape[0]))
validate_predictions(job_to_test, job_to_test['pred_userJobNode'])

1518 / 18224
the mean is 196.92845854221508
r_2 is 0.9416336653501932
RMSE is 126.41804312276199
NRMSE is 0.6419490817050297


In [63]:
job_to_test[['real_pow','pred_userJobNode']]

,real_pow,pred_userJobNode
job_id,,
309244,30.570750,183.006725
309361,114.902552,111.967869
309416,112.969828,113.377266
309418,114.878060,107.127820
309425,85.847301,86.722285
309430,464.197466,474.514831
309431,263.168530,217.925976
309533,610.471164,283.323676
309547,542.963279,201.786415
